# Init
This notebooks initializes the functions that allow us to import other notebooks as modules. We use this method, because it allows us to create and import functions with IPython syntax (useful to run functions with %R magic).

It also sets some plotting defaults, which probabaly would be better done elsewhere.

In [ ]:
# This allows us to run scripts from the src directory.
module_path = os.path.abspath(os.path.join('/Users/hilmarzech/Projects/WP1/analysis/src')) # This path might not work on windows machines, might need '\\' instead of '/'
if module_path not in sys.path:
    sys.path.append(module_path)
# These imports help reload functions from other notebooks after making changes (e.g, dp=reload(dp))
%load_ext autoreload
%autoreload 2
from importlib import reload
# These imports are required to make other notebooks loadable
import io, os, sys, types, random
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
# Some setup to make matlab plots look better
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(rc={'figure.figsize':(11,6)})
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
from wp1 import pandas_extensions
@pd.api.extensions.register_dataframe_accessor("utils") # Extending dataframe functionality
class DataFrameExtensions(pandas_extensions.DataFrameExtensions):
    pass
@pd.api.extensions.register_series_accessor("utils") # Extending dataframe functionality
class SeriesExtensions(pandas_extensions.SeriesExtensions):
    pass

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        #print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]



In [ ]:
sys.meta_path.append(NotebookFinder())